# Работа на семинаре 29.05.2021

## models.py

In [1]:
from sqlalchemy import (
    Column,
    Integer,
    String,
    DateTime,
    Float,
    ForeignKey,
    CheckConstraint
)
from sqlalchemy.ext.declarative import as_declarative, declared_attr

__all__ = [
    'Base',
    'City',
    'ExamMark',
    'Lecturer',
    'Student',
    'Subject',
    'SubjectLecturer',
    'University'
]

CASCADE = "CASCADE"
SHORT_VARCHAR = 80
MIDDLE_VARCHAR = 255


@as_declarative()
class Base:
    id = Column(Integer, primary_key=True, autoincrement=True)

    @declared_attr
    def __tablename__(self):
        table_name = ''
        ascii_upper = range(ord('A'), ord('Z') + 1)
        for i, s in enumerate(self.__name__):
            if ord(s) in ascii_upper and i != 0:
                table_name += '_'
            table_name += s.upper()

        return table_name + 'S'

    def __repr__(self):
        field_strings = []
        for attr in self.__repr_attrs__:
            field_strings.append(f'{attr}={getattr(self, attr)}')

        return f"<{self.__class__.__name__} ({', '.join(field_strings)})>"

    __str__ = __repr__
    __repr_attrs__ = ['id']


class City(Base):
    name = Column(String(SHORT_VARCHAR))

    __tablename__ = "CITIES"
    __repr_attrs__ = ['name']


class ExamMark(Base):
    mark = Column(Integer)
    date = Column(DateTime)
    student_id = Column(Integer, ForeignKey("STUDENTS.id", ondelete=CASCADE))
    subject_id = Column(Integer, ForeignKey("SUBJECTS.id", ondelete=CASCADE))

    __repr_attrs__ = ['mark', 'date', 'student_id', 'subject_id']


class Lecturer(Base):
    first_name = Column(String(SHORT_VARCHAR))
    last_name = Column(String(SHORT_VARCHAR))
    city = Column(String(SHORT_VARCHAR))
    university_id = Column(Integer, ForeignKey("UNIVERSITIES.id", ondelete=CASCADE))

    __repr_attrs__ = ['first_name', 'last_name', 'city', 'university_id']


class Student(Base):
    first_name = Column(String(SHORT_VARCHAR))
    last_name = Column(String(SHORT_VARCHAR))
    stipend = Column(Float, CheckConstraint("stipend >= 0.00", name="stipend_is_positive"))
    course = Column(Integer)
    city = Column(String(SHORT_VARCHAR))
    birthdate = Column(DateTime)
    university_id = Column(Integer, ForeignKey("UNIVERSITIES.id", ondelete=CASCADE))

    __repr_attrs__ = ['first_name', 'last_name', 'stipend', 'course', 'city', 'birthdate', 'university_id']


class SubjectLecturer(Base):
    lecturer_id = Column(Integer, ForeignKey("LECTURERS.id", ondelete=CASCADE))
    subject_id = Column(Integer, ForeignKey("SUBJECTS.id", ondelete=CASCADE))

    __tablename__ = "SUBJECTS_LECTURERS"
    __repr_attrs__ = ['lecturer_id', 'subject_id']


class Subject(Base):
    name = Column(String(SHORT_VARCHAR), index=True)
    hour = Column(Integer, index=True)
    semester = Column(Integer)

    __repr_attrs__ = ['name', 'hour', 'semester']


class University(Base):
    name = Column(String(SHORT_VARCHAR))
    rating = Column(Integer)
    city = Column(String(SHORT_VARCHAR))

    __tablename__ = "UNIVERSITIES"
    __repr_attrs__ = ['name', 'rating', 'city']


## database.py

In [9]:
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from sqlalchemy.orm.session import sessionmaker, Session
from sqlalchemy.sql import Select


class DataBase:
    DATABASE = "sqlite"
    DB_NAME = "resources/students.db"
    engine: Engine = create_engine(f"{DATABASE}:///{DB_NAME}", echo=False)
    session: Session = sessionmaker(bind=engine)()

    @staticmethod
    def fetch_all(statement: Select) -> list:
        with DataBase.engine.connect() as connection:
            return connection.execute(statement).all()

    @staticmethod
    def fetch_one(statement: Select):
        with DataBase.engine.connect() as connection:
            return connection.execute(statement).one()

    @staticmethod
    def fetch_scalar(statement: Select):
        with DataBase.engine.connect() as connection:
            return connection.execute(statement).scalar_one()

    @staticmethod
    def fetch_scalars(statement: Select):
        with DataBase.engine.connect() as connection:
            return connection.execute(statement).scalars().all()

    
# Base.metadata.create_all(DataBase.engine)


## utils.py

In [10]:
import re


def task(n):
    def wrapper(function):
        def decorator(*args, **kwargs):
            print(str(n) + ". " + re.sub(r' +', ' ', function.__doc__.strip()) + "\n")
            result = function(*args, **kwargs)
#             print('\n\n')
            return result
        return decorator
    return wrapper

## queries.py

In [11]:
from datetime import datetime
from pprint import pprint

from sqlalchemy import cast, Integer
from sqlalchemy.sql import select, Select, or_, func

db = DataBase()

In [12]:
@task("2")
def second():
    """
    Напишите запрос, позволяющий получить из таблицы exam_marks
    значения столбца mark (экзаменационная оценка) для всех студентов,
    исключив из списка повторение одинаковых строк.
    Результат не должен содержать значений (None).
    Упорядочить результат по возрастанию значения оценки.
    """
    stmt: Select = select(
        ExamMark.mark
    ).distinct().where(
        ExamMark.mark.isnot(None)
    ).order_by(
        ExamMark.mark
    )

    pprint(db.fetch_scalars(stmt))

    
second()

2. Напишите запрос, позволяющий получить из таблицы exam_marks
 значения столбца mark (экзаменационная оценка) для всех студентов,
 исключив из списка повторение одинаковых строк.
 Результат не должен содержать значений (None).
 Упорядочить результат по возрастанию значения оценки.

[1, 2, 3, 4, 5]


In [13]:
@task("3")
def third():
    """
    Напишите запрос для получения списка студентов без определенного места жительства.
    Результат должен содержать идентификатор студента, фамилию, имя.
    """
    stmt: Select = select(
        Student.id, Student.last_name, Student.first_name
    ).where(
        Student.city.is_(None)
    )

    pprint(db.fetch_all(stmt))


third() 

3. Напишите запрос для получения списка студентов без определенного места жительства.
 Результат должен содержать идентификатор студента, фамилию, имя.

[(63, 'Медведева', 'Ирина'), (64, 'Афанасьева', 'Оксана')]


In [14]:
@task("4")
def fourth():
    """
    Напишите запрос для получения списка студентов,
    проживающих в Воронеже и не получающих стипендию.
    """
    stmt: Select = select(
        Student
    ).where(
        Student.city == 'Воронеж',
        or_(Student.stipend == 0, Student.stipend.is_(None))
    )

    pprint(db.fetch_all(stmt))
    
    
fourth()

4. Напишите запрос для получения списка студентов,
 проживающих в Воронеже и не получающих стипендию.

[(265, 'Андрей', 'Павлов', 0.0, 3, 'Воронеж', datetime.datetime(1979, 11, 5, 0, 0), 10)]


In [15]:
@task("5")
def fifth():
    """
    Напишите запрос для получения списка университетов,
    расположенных в Москве и имеющих рейтинг меньший, чем у НГУ.
    Значение рейтинга НГУ получите с помощью отдельного запроса или подзапроса.
    """
    ngu_rating = db.fetch_scalar(select(
        University.rating
    ).where(
        University.name == 'НГУ'
    ))

    stmt: Select = select(
        University
    ).where(
        University.city == 'Москва',
        University.rating < ngu_rating
    )

    pprint(db.fetch_all(stmt))
    
    
fifth()

5. Напишите запрос для получения списка университетов,
 расположенных в Москве и имеющих рейтинг меньший, чем у НГУ.
 Значение рейтинга НГУ получите с помощью отдельного запроса или подзапроса.

[(44, 'ФинУ', 330, 'Москва'), (49, 'МТУСИ', 295, 'Москва')]


In [16]:
@task("6")
def sixth():
    """
    Напишите запрос, выполняющий вывод находящихся в таблице
    EXAM_MARKS номеров предметов обучения, экзамены по
    которым сдавались между 1 и 21 марта 2020 г.
    """
    stmt: Select = select(
        ExamMark.subject_id, ExamMark.date
    ).where(
        ExamMark.date.between(datetime(2020, 3, 1), datetime(2020, 3, 21))
    )

    pprint(db.fetch_all(stmt))
    
    
sixth()

6. Напишите запрос, выполняющий вывод находящихся в таблице
 EXAM_MARKS номеров предметов обучения, экзамены по
 которым сдавались между 1 и 21 марта 2020 г.

[(73, datetime.datetime(2020, 3, 20, 0, 0)),
 (12, datetime.datetime(2020, 3, 13, 0, 0)),
 (10, datetime.datetime(2020, 3, 21, 0, 0))]


In [17]:
@task("7")
def seventh():
    """
    Напишите запрос, который выполняет вывод названий
    предметов обучения, начинающихся на букву 'И'.
    """
    stmt: Select = select(
        Subject.name
    ).where(
        Subject.name.startswith('И')
    )

    pprint(db.fetch_all(stmt))
    
    
seventh()

7. Напишите запрос, который выполняет вывод названий
 предметов обучения, начинающихся на букву 'И'.

[('Информатика',), ('История',)]


In [18]:
@task("8")
def eighth():
    """
    Напишите запрос, выбирающий сведения о студентах,
    у которых имена начинаются на букву 'И' или 'С'.
    """
    stmt: Select = select(
        Student
    ).where(
        or_(Student.first_name.startswith('И'), Student.first_name.startswith('С'))
    )

    pprint(db.fetch_all(stmt))
    
    
eighth()

8. Напишите запрос, выбирающий сведения о студентах,
 у которых имена начинаются на букву 'И' или 'С'.

[(1, 'Иван', 'Иванов', 150.0, 1, 'Орел', datetime.datetime(1982, 12, 3, 0, 0), 10),
 (59, 'Софья', 'Дементьева', 150.0, 1, 'Нерчинск', datetime.datetime(1996, 9, 25, 0, 0), 15),
 (61, 'Ия', 'Соколова', 100.0, 1, 'Колпашево', datetime.datetime(1995, 6, 25, 0, 0), 18),
 (63, 'Ирина', 'Медведева', 100.0, 1, None, datetime.datetime(2000, 8, 22, 0, 0), 15),
 (71, 'Синицин', 'Кондрат', 200.0, 4, 'Ишим', datetime.datetime(1995, 2, 23, 0, 0), 18),
 (77, 'Светлана', 'Мишина', 150.0, 2, 'Рыльск', datetime.datetime(1997, 5, 22, 0, 0), 18),
 (83, 'Ирина', 'Сорокина', 200.0, 3, 'Усмань', datetime.datetime(1993, 9, 28, 0, 0), 48),
 (88, 'Ирина', 'Жданова', 200.0, 3, 'Тверь', datetime.datetime(2002, 3, 2, 0, 0), 10),
 (91, 'Ираида', 'Блохина', 250.0, 3, 'Новый Уренгой', datetime.datetime(1997, 3, 23, 0, 0), 22),
 (92, 'Светлана', 'Мельникова', 250.0, 4, 'Кропоткин (Краснод.)', datetime.datetime(199

In [19]:
@task("9")
def ninth():
    """
    Напишите запрос для получения списка предметов обучения,
    названия которых состоят из более одного слова.
    """
    stmt: Select = select(
        Subject
    ).where(
        Subject.name.like('% %')
    )

    pprint(db.fetch_all(stmt))
    
    
ninth()

9. Напишите запрос для получения списка предметов обучения,
 названия которых состоят из более одного слова.

[(12, 'Анализ данных', 42, 1)]


In [20]:
@task("10")
def tenth():
    """
    Напишите запрос для получения списка студентов,
    фамилии которых состоят из трех букв.
    """
    stmt: Select = select(
        Student
    ).where(
        func.length(Student.last_name) == 3
    )

    pprint(db.fetch_all(stmt))
    
    
tenth()

10. Напишите запрос для получения списка студентов,
 фамилии которых состоят из трех букв.

[(655, 'Бернар', 'Шоу', 120.0, 2, 'Ульяновск', datetime.datetime(1979, 8, 5, 0, 0), 44),
 (656, 'Джон', 'Доу', 120.0, 2, 'Ульяновск', datetime.datetime(1979, 8, 5, 0, 0), 44)]


In [21]:
@task("11")
def eleventh():
    """
    Составьте запрос для таблицы STUDENT таким образом,
    чтобы получить результат в следующем виде.
    Распечатайте первые 9 записей результата.

    И. Иванов   1982-12-03
    П. Петров   1980-12-01
    В. Сидоров  1979-06-07
    """
    stmt: Select = select(
        Student.first_name, Student.last_name, Student.birthdate
    ).limit(9)

    pprint([f"{i[0][0]}. {i[1]}   {datetime.strftime(i[2], '%Y-%m-%d')}" for i in db.fetch_all(stmt)])


eleventh()

11. Составьте запрос для таблицы STUDENT таким образом,
 чтобы получить результат в следующем виде.
 Распечатайте первые 9 записей результата.

 И. Иванов 1982-12-03
 П. Петров 1980-12-01
 В. Сидоров 1979-06-07

['И. Иванов   1982-12-03',
 'П. Петров   1980-12-01',
 'В. Сидоров   1979-06-07',
 'Б. Кузнецов   1981-12-08',
 'О. Зайцева   1981-05-01',
 'П. Котов   2021-02-28',
 'В. Белкин   1980-01-07',
 'Е. Сергеева   1997-07-04',
 'В. Кудряшова   2002-02-18']


In [22]:
@task("12")
def twelve():
    """
    Напишите запрос для получения списка студентов,
    фамилии которых начинаются на ‘Ков’ или на ‘Куз’.
    """
    stmt: Select = select(
        Student
    ).where(
        or_(Student.last_name.startswith('Ков'), Student.last_name.startswith('Куз'))
    )

    pprint(db.fetch_all(stmt))
    
    
twelve()

12. Напишите запрос для получения списка студентов,
 фамилии которых начинаются на ‘Ков’ или на ‘Куз’.

[(10, 'Борис', 'Кузнецов', 0.0, 2, 'Брянск', datetime.datetime(1981, 12, 8, 0, 0), 10),
 (131, 'Ефим', 'Ковалев', 200.0, 2, 'Волгоград', datetime.datetime(1995, 11, 28, 0, 0), 47)]


In [23]:
@task("13")
def thirteenth():
    """
    Напишите запрос для получения списка предметов,
    названия которых оканчиваются на ‘ия’.
    """
    stmt: Select = select(
        Subject
    ).where(
        Subject.name.endswith('ия')
    )

    pprint(db.fetch_all(stmt))
    

thirteenth()

13. Напишите запрос для получения списка предметов,
 названия которых оканчиваются на ‘ия’.

[(56, 'История', 34, 4)]


In [24]:
@task("14")
def fourteenth():
    """
    Составьте запрос, выводящий фамилии, имена студентов и величину получаемых ими стипендий,
    при этом значения стипендий должны быть увеличены в 100 раз.
    Распечатайте первые 10 результатов.
    """
    stmt: Select = select(
        Student.last_name, Student.first_name, Student.stipend * 100
    ).limit(10)

    pprint(db.fetch_all(stmt))


fourteenth()

14. Составьте запрос, выводящий фамилии, имена студентов и величину получаемых ими стипендий,
 при этом значения стипендий должны быть увеличены в 100 раз.
 Распечатайте первые 10 результатов.

[('Иванов', 'Иван', 15000.0),
 ('Петров', 'Петр', 20000.0),
 ('Сидоров', 'Вадим', 15000.0),
 ('Кузнецов', 'Борис', 0.0),
 ('Зайцева', 'Ольга', 25000.0),
 ('Котов', 'Павел', 15000.0),
 ('Белкин', 'Вадим', 25000.0),
 ('Сергеева', 'Елизавета', 15000.0),
 ('Кудряшова', 'Вера', 10000.0),
 ('Журавлева', 'Вера', 0.0)]


In [25]:
@task("15")
def fifteenth():
    """
    Составьте запрос для таблицы UNIVERSITY таким образом, чтобы выходная таблица
    содержала всего один столбец в следующем виде: Код-10; ВГУ-г.ВОРОНЕЖ; Рейтинг=296.
    """
    stmt: Select = select(
        University.id, University.name, University.city, University.rating
    )

    pprint([f"Код-{i[0]}; {i[1]}-г.{i[2].upper()}; Рейтинг={i[3]}." for i in db.fetch_all(stmt)])


fifteenth()

15. Составьте запрос для таблицы UNIVERSITY таким образом, чтобы выходная таблица
 содержала всего один столбец в следующем виде: Код-10; ВГУ-г.ВОРОНЕЖ; Рейтинг=296.

['Код-10; ВГУ-г.ВОРОНЕЖ; Рейтинг=296.',
 'Код-11; НГУ-г.НОВОСИБИРСК; Рейтинг=345.',
 'Код-14; БГУ-г.БЕЛГОРОД; Рейтинг=326.',
 'Код-15; ТГУ-г.ТОМСК; Рейтинг=368.',
 'Код-18; ВГМА-г.ВОРОНЕЖ; Рейтинг=327.',
 'Код-22; МГУ-г.МОСКВА; Рейтинг=400.',
 'Код-32; РГУ-г.РОСТОВ; Рейтинг=416.',
 'Код-44; ФинУ-г.МОСКВА; Рейтинг=330.',
 'Код-45; МГТУ-г.МОСКВА; Рейтинг=372.',
 'Код-46; Политех-г.САНКТ-ПЕТЕРБУРГ; Рейтинг=300.',
 'Код-47; КФУ-г.КАЗАНЬ; Рейтинг=330.',
 'Код-48; УЛГУ-г.УЛЬЯНОВСК; Рейтинг=231.',
 'Код-49; МТУСИ-г.МОСКВА; Рейтинг=295.']


In [26]:
@task("16")
def sixteenth():
    """
    Напишите запрос для подсчета количества студентов,
    сдававших экзамен по предмету обучения с идентификатором 10.
    """
    stmt: Select = select(
        func.count(ExamMark.subject_id)
    ).where(
        ExamMark.subject_id == 10
    )

    pprint(db.fetch_scalar(stmt))


sixteenth()

16. Напишите запрос для подсчета количества студентов,
 сдававших экзамен по предмету обучения с идентификатором 10.

10


In [27]:
@task("17")
def seventeenth():
    """
    Напишите запрос, который позволяет подсчитать в таблице
    EXAM_MARKS количество различных предметов обучения.
    """
    stmt: Select = select(
        ExamMark.subject_id
    ).distinct()

    pprint(len(db.fetch_all(stmt)))

    
seventeenth()

17. Напишите запрос, который позволяет подсчитать в таблице
 EXAM_MARKS количество различных предметов обучения.

9


In [28]:
@task("18")
def eighteenth():
    """
    Напишите запрос, который для каждого студента выполняет
    выборку его идентификатора и минимальной из полученных им оценок.
    """
    stmt: Select = select(
        ExamMark.student_id, func.min(ExamMark.mark)
    ).group_by(
        ExamMark.student_id
    )

    pprint(db.fetch_all(stmt))
    
    
eighteenth()

18. Напишите запрос, который для каждого студента выполняет
 выборку его идентификатора и минимальной из полученных им оценок.

[(1, 1),
 (6, 4),
 (10, 5),
 (12, 5),
 (15, 5),
 (32, 4),
 (55, 5),
 (62, 4),
 (64, 4),
 (65, 1),
 (71, 3),
 (73, 1),
 (76, 2),
 (77, 1),
 (79, 1),
 (82, 5),
 (83, 1),
 (85, 1),
 (88, 5),
 (90, 4),
 (91, 5),
 (92, 5),
 (97, 1),
 (99, 5),
 (101, 2),
 (103, 2),
 (105, 3),
 (110, 1),
 (116, 1),
 (117, 4),
 (123, 2),
 (126, 1),
 (128, 3),
 (149, 2),
 (203, 3)]


In [29]:
@task("19")
def nineteenth():
    """
    Напишите запрос, который для каждого конкретного дня сдачи
    экзамена выводит данные о количестве студентов,
    сдававших экзамен в этот день.
    """
    stmt: Select = select(
        ExamMark.date, func.count(ExamMark.student_id)
    ).group_by(
        ExamMark.date
    )

    pprint(db.fetch_all(stmt))


nineteenth()

19. Напишите запрос, который для каждого конкретного дня сдачи
 экзамена выводит данные о количестве студентов,
 сдававших экзамен в этот день.

[(datetime.datetime(1999, 6, 17, 0, 0), 1),
 (datetime.datetime(1999, 6, 22, 0, 0), 1),
 (datetime.datetime(2000, 1, 5, 0, 0), 1),
 (datetime.datetime(2000, 1, 18, 0, 0), 1),
 (datetime.datetime(2000, 1, 23, 0, 0), 1),
 (datetime.datetime(2006, 1, 12, 0, 0), 1),
 (datetime.datetime(2019, 5, 11, 0, 0), 1),
 (datetime.datetime(2019, 6, 8, 0, 0), 1),
 (datetime.datetime(2019, 7, 4, 0, 0), 1),
 (datetime.datetime(2019, 7, 6, 0, 0), 1),
 (datetime.datetime(2019, 7, 24, 0, 0), 1),
 (datetime.datetime(2019, 8, 14, 0, 0), 1),
 (datetime.datetime(2019, 8, 30, 0, 0), 1),
 (datetime.datetime(2019, 9, 2, 0, 0), 1),
 (datetime.datetime(2019, 10, 28, 0, 0), 1),
 (datetime.datetime(2019, 12, 17, 0, 0), 1),
 (datetime.datetime(2019, 12, 25, 0, 0), 1),
 (datetime.datetime(2019, 12, 31, 0, 0), 1),
 (datetime.datetime(2020, 2, 10, 0, 0), 1),
 (datetime.datetime(

In [30]:
@task("20")
def twentieth():
    """
    Напишите запрос, выдающий идентификатор студента и его средний балл.
    """
    stmt: Select = select(
        ExamMark.student_id, func.avg(ExamMark.mark)
    ).group_by(
        ExamMark.student_id
    )

    pprint(db.fetch_all(stmt))
    

twentieth()

20. Напишите запрос, выдающий идентификатор студента и его средний балл.

[(1, 1.0),
 (6, 4.0),
 (10, 5.0),
 (12, 5.0),
 (15, 5.0),
 (32, 4.0),
 (55, 5.0),
 (62, 4.0),
 (64, 4.0),
 (65, 1.0),
 (71, 3.0),
 (73, 1.0),
 (76, 2.0),
 (77, 1.0),
 (79, 1.0),
 (82, 5.0),
 (83, 1.0),
 (85, 2.5),
 (88, 5.0),
 (90, 4.0),
 (91, 5.0),
 (92, 5.0),
 (97, 1.0),
 (99, 5.0),
 (101, 2.0),
 (103, 2.0),
 (105, 3.0),
 (110, 1.0),
 (116, 1.0),
 (117, 4.0),
 (123, 2.0),
 (126, 1.0),
 (128, 4.0),
 (149, 2.0),
 (203, 3.0)]


In [32]:
@task("21")
def twenty_first():
    """
    Напишите запрос, выдающий средний балл для каждого экзамена.
    """
    stmt: Select = select(
        Subject, func.avg(ExamMark.mark)
    ).join(Subject).group_by(
        ExamMark.subject_id
    )

    pprint(db.fetch_all(stmt))


twenty_first()    

21. Напишите запрос, выдающий средний балл для каждого экзамена.

[(10, 'Информатика', 56, 1, 4.0),
 (11, 'Программирование', 34, 2, 1.0),
 (12, 'Анализ данных', 42, 1, 2.0),
 (13, 'ОБЖ', 34, 2, 2.5),
 (22, 'Физика', 34, 1, 2.8333333333333335),
 (43, 'Математика', 56, 2, 2.0),
 (56, 'История', 34, 4, 3.1666666666666665),
 (73, 'Физкультура', 34, 5, 3.0),
 (94, 'Английский', 56, 3, 3.6666666666666665)]


In [33]:
@task("22")
def twenty_second():
    """
    Напишите запрос, определяющий количество сдававших
    студентов для каждого предмета, по которому был экзамен.
    """
    stmt: Select = select(
        Subject, func.count(ExamMark.student_id)
    ).join(ExamMark).group_by(
        ExamMark.subject_id
    )

    pprint(db.fetch_all(stmt))


twenty_second()

22. Напишите запрос, определяющий количество сдававших
 студентов для каждого предмета, по которому был экзамен.

[(10, 'Информатика', 56, 1, 10),
 (11, 'Программирование', 34, 2, 1),
 (12, 'Анализ данных', 42, 1, 5),
 (13, 'ОБЖ', 34, 2, 4),
 (22, 'Физика', 34, 1, 6),
 (43, 'Математика', 56, 2, 2),
 (56, 'История', 34, 4, 6),
 (73, 'Физкультура', 34, 5, 4),
 (94, 'Английский', 56, 3, 3)]


In [34]:
@task("23")
def twenty_third():
    """
    Напишите запрос для определения количества предметов,
    изучаемых на каждом курсе.
    """
    course = cast(Subject.semester + 1, Integer) / 2
    stmt: Select = select(
        course, func.count(Subject.id)
    ).group_by(
        course
    )

    pprint(db.fetch_all(stmt))


twenty_third()

23. Напишите запрос для определения количества предметов,
 изучаемых на каждом курсе.

[(1, 6), (2, 2), (3, 1)]


In [35]:
@task("24")
def twenty_fourth():
    """
    Для каждого университета напишите запрос,
    выводящий суммарную стипендию обучающихся в нем студентов,
    с последующей сортировкой списка по этому значению.
    """
    stmt: Select = select(
        University, func.sum(Student.stipend)
    ).join(Student).group_by(
        Student.university_id
    )

    pprint(db.fetch_all(stmt))


twenty_fourth()

24. Для каждого университета напишите запрос,
 выводящий суммарную стипендию обучающихся в нем студентов,
 с последующей сортировкой списка по этому значению.

[(10, 'ВГУ', 296, 'Воронеж', 3250.0),
 (11, 'НГУ', 345, 'Новосибирск', 400.0),
 (14, 'БГУ', 326, 'Белгород', 1700.0),
 (15, 'ТГУ', 368, 'Томск', 650.0),
 (18, 'ВГМА', 327, 'Воронеж', 1000.0),
 (22, 'МГУ', 400, 'Москва', 2180.0),
 (32, 'РГУ', 416, 'Ростов', 1050.0),
 (44, 'ФинУ', 330, 'Москва', 2220.0),
 (45, 'МГТУ', 372, 'Москва', 650.0),
 (46, 'Политех', 300, 'Санкт-Петербург', 1250.0),
 (47, 'КФУ', 330, 'Казань', 1450.0),
 (48, 'УЛГУ', 231, 'Ульяновск', 1200.0)]


In [36]:
@task("25")
def twenty_fifth():
    """
    Для каждого студента напишите запрос, выводящий идентификатор студента
    и среднее значение оценок, полученных им на всех экзаменах.
    """
    stmt: Select = select(
        ExamMark.student_id, func.avg(ExamMark.mark)
    ).group_by(
        ExamMark.student_id
    )

    pprint(db.fetch_all(stmt))
    
    
twenty_fifth()

25. Для каждого студента напишите запрос, выводящий идентификатор студента
 и среднее значение оценок, полученных им на всех экзаменах.

[(1, 1.0),
 (6, 4.0),
 (10, 5.0),
 (12, 5.0),
 (15, 5.0),
 (32, 4.0),
 (55, 5.0),
 (62, 4.0),
 (64, 4.0),
 (65, 1.0),
 (71, 3.0),
 (73, 1.0),
 (76, 2.0),
 (77, 1.0),
 (79, 1.0),
 (82, 5.0),
 (83, 1.0),
 (85, 2.5),
 (88, 5.0),
 (90, 4.0),
 (91, 5.0),
 (92, 5.0),
 (97, 1.0),
 (99, 5.0),
 (101, 2.0),
 (103, 2.0),
 (105, 3.0),
 (110, 1.0),
 (116, 1.0),
 (117, 4.0),
 (123, 2.0),
 (126, 1.0),
 (128, 4.0),
 (149, 2.0),
 (203, 3.0)]


In [37]:
@task("26")
def twenty_sixth():
    """
    Напишите запрос, выводящий количество студентов, проживающих в каждом городе.
    Список отсортировать в порядке убывания количества студентов.
    """
    stmt: Select = select(
        Student.city, func.count(Student.id)
    ).group_by(Student.city)

    pprint(db.fetch_all(stmt))
    
    
twenty_sixth()

26. Напишите запрос, выводящий количество студентов, проживающих в каждом городе.
 Список отсортировать в порядке убывания количества студентов.

[(None, 2),
 ('Ангарск', 1),
 ('Архангельск', 1),
 ('Байкальск', 1),
 ('Белгород', 2),
 ('Беломорск', 1),
 ('Благовещенск (Амур.)', 1),
 ('Брянск', 2),
 ('Валдай', 1),
 ('Верхний Баскунчак', 1),
 ('Верхоянск', 1),
 ('Владикавказ', 1),
 ('Волгоград', 1),
 ('Вологда', 1),
 ('Волоколамск', 1),
 ('Воронеж', 3),
 ('Геленджик', 1),
 ('Грозный', 2),
 ('Дербент', 1),
 ('Джубга', 2),
 ('Екатеринбург', 1),
 ('Енисейск', 1),
 ('Жуковский', 1),
 ('Забайкальск', 1),
 ('Златоуст', 1),
 ('Иваново', 1),
 ('Игарка', 1),
 ('Иркутск', 2),
 ('Ишим', 1),
 ('Калининград', 1),
 ('Каменск-Уральский', 1),
 ('Камень-на-Оби', 1),
 ('Канск', 1),
 ('Кемерово', 1),
 ('Кизилюрт', 1),
 ('Кисловодск', 1),
 ('Колпашево', 1),
 ('Корсаков', 3),
 ('Кропоткин (Краснод.)', 2),
 ('Курганинск', 1),
 ('Курск', 3),
 ('Кущевская', 1),
 ('Кыштым', 1),
 ('Липецк', 2),
 ('Лянтор', 1),
 ('

In [38]:
@task("extra_students")
def extra_students():
    """
    Подзапрос для получения всех студентов,
    обучающихся в своем городе
    """
    sub: Select = select(
        University.city
    ).where(
        University.id == Student.university_id
    ).scalar_subquery()

    stmt: Select = select(
        Student
    ).where(
        Student.city == sub
    )

    pprint(db.fetch_all(stmt))
    
    
extra_students()

extra_students. Подзапрос для получения всех студентов,
 обучающихся в своем городе

[(6, 'Вадим', 'Сидоров', 150.0, 4, 'Москва', datetime.datetime(1979, 6, 7, 0, 0), 22),
 (32, 'Павел', 'Котов', 150.0, 5, 'Белгород', datetime.datetime(2021, 2, 28, 0, 0), 14),
 (55, 'Вадим', 'Белкин', 250.0, 5, 'Воронеж', datetime.datetime(1980, 1, 7, 0, 0), 10),
 (200, 'Александр', 'Баранов', 220.0, 2, 'Москва', datetime.datetime(2001, 8, 18, 0, 0), 44),
 (203, 'Екатерина', 'Петрова', 250.0, 4, 'Белгород', datetime.datetime(1999, 5, 21, 0, 0), 14),
 (207, 'Федор', 'Миронова', 200.0, 3, 'Москва', datetime.datetime(2002, 4, 15, 0, 0), 45),
 (265, 'Андрей', 'Павлов', 0.0, 3, 'Воронеж', datetime.datetime(1979, 11, 5, 0, 0), 10),
 (654, 'Артем', 'Лукин', 200.0, 3, 'Воронеж', datetime.datetime(1981, 12, 1, 0, 0), 10)]


In [39]:
@task("extra_lecturers")
def extra_lecturers():
    """
    Подзапрос для получения информации об преподавателях,
    работающих не в своем городе
    """
    sub: Select = select(
        University.city
    ).where(
        University.id == Lecturer.university_id
    ).scalar_subquery()

    stmt: Select = select(
        Lecturer
    ).where(
        Lecturer.city != sub
    )

    pprint(db.fetch_all(stmt))
    
    
extra_lecturers()

extra_lecturers. Подзапрос для получения информации об преподавателях,
 работающих не в своем городе

[(1, 'Лукия', 'Сафонова', 'Смоленск', 46),
 (3, 'Акулина', 'Мамонтова', 'Троицк (Моск.)', 11),
 (4, 'Оксана', 'Богданова', 'Всеволожск', 18),
 (5, 'Нинель', 'Брагина', 'Кижи', 48),
 (6, 'Иванна', 'Максимова', 'Сковородино', 10),
 (7, 'Оксана', 'Дмитриева', 'Шарья', 48),
 (8, 'Светлана', 'Ситникова', 'Сорочинск', 18),
 (9, 'Людмила', 'Крюкова', 'Анива', 18),
 (10, 'Майя', 'Артемьева', 'Курск', 18),
 (11, 'Ираида', 'Куликова', 'Городовиковск', 47),
 (12, 'Анна', 'Кудрявцева', 'Петушки', 47),
 (13, 'Оксана', 'Зыкова', 'Калач', 10),
 (14, 'Руслан', 'Стрелков', 'Королев', 46),
 (15, 'Прокофий', 'Дорофеев', 'Санкт-Петербург', 32),
 (16, 'Михаил', 'Волков', 'Черусти', 45),
 (17, 'Будимир', 'Сафонов', 'Киржач', 18),
 (18, 'Юлиан', 'Зимин', 'Черский', 46),
 (19, 'Трофим', 'Алексеев', 'Мценск', 10),
 (20, 'Ладимир', 'Тихонов', 'Кропоткин (Краснод.)', 15),
 (21, 'Виссарион', 'Елисеев', 'Курганинс